# Queries in PyTables

> Objectives:
> * Query HDF5 files without loading them in-memory
> * How to query normalized and denormalized tables
> * Index columns in tables for accelerating queries

In [24]:
import os
import numpy as np
import pandas as pd
import tables

In [25]:
%ls -lh structuring compression

compression:
total 326032
-rw-r--r--  1 albertofernandezmartinez  staff    17M May 19 16:12 blosc-1-shuffle.h5
-rw-r--r--  1 albertofernandezmartinez  staff   5.0M May 19 16:12 blosc-5-shuffle.h5
-rw-r--r--  1 albertofernandezmartinez  staff   5.3M May 19 16:07 blosc-9-shuffle.h5
-rw-r--r--  1 albertofernandezmartinez  staff   5.6M May 19 16:12 blosc-lz4-1-shuffle.h5
-rw-r--r--  1 albertofernandezmartinez  staff   5.4M May 19 16:12 blosc-lz4-5-shuffle.h5
-rw-r--r--  1 albertofernandezmartinez  staff   5.0M May 19 16:07 blosc-lz4-9-shuffle.h5
-rw-r--r--  1 albertofernandezmartinez  staff   5.1M May 19 16:12 blosc-lz4hc-1-shuffle.h5
-rw-r--r--  1 albertofernandezmartinez  staff   4.8M May 19 16:12 blosc-lz4hc-5-shuffle.h5
-rw-r--r--  1 albertofernandezmartinez  staff   4.7M May 19 16:07 blosc-lz4hc-9-shuffle.h5
-rw-r--r--  1 albertofernandezmartinez  staff    17M May 19 16:12 blosc-snappy-1-shuffle.h5
-rw-r--r--  1 albertofernandezmartinez  staff    17M May 19 16:12 blosc-snappy-5-shuffl

## Querying in PyTables

### Denormalized tables

In [26]:
h5denorm = "structuring/no-compressed.h5"
h5file = tables.open_file(h5denorm)
h5lens = h5file.root.lens

In [27]:
h5lens

/lens (Table(1000209,)) ''
  description := {
  "user_id": Int32Col(shape=(), dflt=0, pos=0),
  "rating": Int8Col(shape=(), dflt=0, pos=1),
  "unix_timestamp": Int64Col(shape=(), dflt=0, pos=2),
  "title": StringCol(itemsize=100, shape=(), dflt='', pos=3),
  "genres": StringCol(itemsize=50, shape=(), dflt='', pos=4)}
  byteorder := 'little'
  chunkshape := (402,)

In [28]:
%%time
ratings = [0] * 6
for rt in range(0,6):
    ratings[rt] = sum(1 for r in h5lens.where("(title == b'Tom and Huck (1995)') & (rating == rt)"))

CPU times: user 537 ms, sys: 442 ms, total: 979 ms
Wall time: 1.05 s


In [29]:
ratings

[0, 4, 15, 28, 18, 3]

In [30]:
h5file.close()

Querying denormalized tables is easy as pie.  Let's see how to manage normalized ones.

### Normalized tables

In [31]:
h5norm = "compression/blosc-zstd-5-shuffle.h5"
h5file = tables.open_file(h5norm)
h5ratings = h5file.root.ratings
h5movies = h5file.root.movies

In [32]:
h5ratings

/ratings (Table(1000209,), shuffle, blosc:zstd(5)) ''
  description := {
  "user_id": Int32Col(shape=(), dflt=0, pos=0),
  "movie_id": Int32Col(shape=(), dflt=0, pos=1),
  "rating": Int8Col(shape=(), dflt=0, pos=2),
  "unix_timestamp": Int64Col(shape=(), dflt=0, pos=3)}
  byteorder := 'little'
  chunkshape := (7710,)

In [33]:
h5movies

/movies (Table(3883,), shuffle, blosc:zstd(5)) ''
  description := {
  "movie_id": Int32Col(shape=(), dflt=0, pos=0),
  "title": StringCol(itemsize=100, shape=(), dflt='', pos=1),
  "genres": StringCol(itemsize=50, shape=(), dflt='', pos=2)}
  byteorder := 'little'
  chunkshape := (425,)

In [34]:
%%time
ratings = [0] * 6
for rt in range(0,6):
    th_movie_id = [r['movie_id'] for r in h5movies.where("(title == b'Tom and Huck (1995)')")][0]
    ratings[rt] = sum(1 for r in h5ratings.where("(movie_id == th_movie_id) & (rating == rt)"))

CPU times: user 318 ms, sys: 52.6 ms, total: 371 ms
Wall time: 351 ms


In [35]:
ratings

[0, 4, 15, 28, 18, 3]

In [36]:
h5file.close()

So, the query in the normalized version is more than 2~3x faster than using the denormalized file.  However, this is just a simple example, and in general experimentation should be done so as to determine the best layout for your data.

## Indexing

Indexing is a general technique for adding data structures that can accelerate queries.  Let's see how PyTables makes use of this.

### Denormalized case

In [37]:
## Copy the original PyTables table into another file
import shutil
h5idx = "movielens-denorm-indexed.h5"
if os.path.exists(h5idx):
    os.unlink(h5idx)
shutil.copyfile(h5denorm, h5idx)

In [38]:
# Open the new file in 'a'ppend mode
h5i = tables.open_file(h5idx, mode="a")

In [39]:
# Create an index for the 'title' column
h5lens = h5i.root.lens
blosc_filter = tables.Filters(complevel=9, complib="blosc")
%time h5lens.cols.title.create_csindex(filters=blosc_filter)

CPU times: user 1.54 s, sys: 403 ms, total: 1.95 s
Wall time: 2.3 s


1000209

In [40]:
%%time
ratings = [0] * 6
for rt in range(0,6):
    ratings[rt] = sum(1 for r in h5lens.where("(title == b'Tom and Huck (1995)') & (rating == rt)"))

CPU times: user 5.95 ms, sys: 824 µs, total: 6.78 ms
Wall time: 7.15 ms


Ok, so this time is 100x less than without using indexing.  What if we index the `rating` column too?

In [41]:
ratings

[0, 4, 15, 28, 18, 3]

In [42]:
# Create an index for the rating column
%time h5lens.cols.rating.create_csindex(filters=blosc_filter)

CPU times: user 332 ms, sys: 87 ms, total: 419 ms
Wall time: 433 ms


1000209

In [43]:
%%time
ratings = [0] * 6
for rt in range(0,6):
    ratings[rt] = sum(1 for r in h5lens.where("(title == b'Tom and Huck (1995)') & (rating == rt)"))

CPU times: user 3.66 ms, sys: 204 µs, total: 3.87 ms
Wall time: 13.2 ms


Ok, so although small, this represents another improvement in performance.

In [44]:
ratings

[0, 4, 15, 28, 18, 3]

In [45]:
h5i.close()

### Normalized case

In [46]:
## Copy the original PyTables table into another file
import shutil
h5idx = "movielens-norm-indexed.h5"
if os.path.exists(h5idx):
    os.unlink(h5idx)
shutil.copyfile(h5norm, h5idx)

In [47]:
# Open the new file in 'a'ppend mode
h5i = tables.open_file(h5idx, mode="a")
h5ratings = h5i.root.ratings
h5movies = h5i.root.movies

In [48]:
# Create an index for the rating column
blosc_filter = tables.Filters(complevel=9, complib="blosc")
%time h5ratings.cols.rating.create_csindex(filters=blosc_filter)

CPU times: user 395 ms, sys: 61.3 ms, total: 456 ms
Wall time: 570 ms


1000209

In [49]:
%%time
ratings = [0] * 6
for rt in range(6):
    th_movie_id = [r['movie_id'] for r in h5movies.where("(title == b'Tom and Huck (1995)')")][0]
    ratings[rt] = sum(1 for r in h5ratings.where("(movie_id == th_movie_id) & (rating == rt)"))

CPU times: user 349 ms, sys: 48.5 ms, total: 398 ms
Wall time: 619 ms


Hmm, in this case indexing the rating column has not served to accelerate the query (at first sight at least).

In [50]:
ratings

[0, 4, 15, 28, 18, 3]

In [51]:
# Create an index for the movie_id column
%time h5ratings.cols.movie_id.create_csindex(filters=blosc_filter)

CPU times: user 388 ms, sys: 40.7 ms, total: 428 ms
Wall time: 551 ms


1000209

In [52]:
%%time
ratings = [0] * 6
for rt in range(6):
    th_movie_id = [r['movie_id'] for r in h5movies.where("(title == b'Tom and Huck (1995)')")][0]
    ratings[rt] = sum(1 for r in h5ratings.where("(movie_id == th_movie_id) & (rating == rt)"))

CPU times: user 22.1 ms, sys: 3.07 ms, total: 25.1 ms
Wall time: 25.1 ms


This time we see a better acceleration in the query, but cannot compete with the query speed for the denormalized case (which is ~10x faster).

In [53]:
ratings

[0, 4, 15, 28, 18, 3]

In [54]:
h5i.close()

In [55]:
%ls -lh movielens*

-rw-r--r--  1 albertofernandezmartinez  staff   160M May 19 16:45 movielens-denorm-indexed.h5
-rw-r--r--  1 albertofernandezmartinez  staff   9.6M May 19 16:45 movielens-norm-indexed.h5

movielens-1m:
total 11472
-rw-r--r--  1 albertofernandezmartinez  staff   5.1K May 19 11:46 README
-rw-r--r--  1 albertofernandezmartinez  staff    56K May 19 11:46 movies.dat.gz
-rw-r--r--  1 albertofernandezmartinez  staff   5.5M May 19 11:46 ratings.dat.gz
-rw-r--r--  1 albertofernandezmartinez  staff    39K May 19 11:46 users.dat.gz


## Exercise

We have not created an index for the title for the normalized case.  Create such an index and determine if there is a noticeable speed-up or not.  Explain why you think that is the case.  Note: the times for a cold query can be **significatively** different from a hot query.